### Instructions
To run the code below:
1. Read the comments, and alter settings if you wish to change the model or layer.
2. Then run all the cells to download models and run the UI.

To operate the interface:
1. In the UI, enter numbers such as "200-212" in the Image Number entry to see generated images.
2. Click on a region of an image and "Add to context" to help define a rule to change.
3. Click "Search" to find other images that are affected by the same rule.
4. Click on a region of an image and "Copy" to choose a new pattern to use.
5. Click on a target region of an image and "Paste" to paste the pattern in a new place.  Clicking around the red box can adjust it.
6. Click "Execute" to see the effects.  "Toggle" compares to the original model, and "Revert" discards the edit.

Editing a model is challenging, because you need to develop an understanding of the way the model organies its rules.  You can build your intuition by using the "Highlight" button to see how the model generalizes regions that you select in the context.

Particular model edits can be saved or loaded as json files; and other geeky details
can be seen in the source code at http://github.com/davidbau/rewriting.  This
research code is not yet optimized for speed, and contributions are welcome.


In [ ]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit 
pip install ninja 2>> install.log
git clone https://github.com/davidbau/rewriting.git tutorial_code 2>> install.log

In [ ]:
try: # set up path
    import google.colab, sys, torch
    sys.path.append('/content/tutorial_code')
    if not torch.cuda.is_available():
        print("Change runtime type to include a GPU.")  
except:
    pass

In [ ]:
from utils import zdataset, show, labwidget
from rewrite import ganrewrite, rewriteapp
import torch, copy, os, json
from torchvision.utils import save_image
import utils.stylegan2, utils.proggan
from utils.stylegan2 import load_seq_stylegan

# Choices: ganname = 'stylegan' or ganname = 'proggan'
ganname = 'stylegan'

# Choices: modelname = 'church' or faces' or 'horse' or 'kitchen'
modelname = 'church'

# layer 6,7,8,9,10 work OK for different things.
# layer 8 is good for trees or domes in churches, and hats on horses
# layer 6 is good for smiles on faces
# layer 10 is good for hair on faces
layernum = 8

# Number of images to sample when gathering statistics.
size = 1000

# Make a directory for caching some data.
layerscheme = 'default'
expdir = 'results/pgw/%s/%s/%s/layer%d' % (ganname, modelname, layerscheme, layernum)
os.makedirs(expdir, exist_ok=True)

# Load (and download) a pretrained GAN
if ganname == 'stylegan':
    model = load_seq_stylegan(modelname, mconv='seq', truncation=0.50)
    Rewriter = ganrewrite.SeqStyleGanRewriter
elif ganname == 'proggan':
    model = utils.proggan.load_pretrained(modelname)
    Rewriter = ganrewrite.ProgressiveGanRewriter
    
# Create a Rewriter object - this implements our method.
zds = zdataset.z_dataset_for_model(model, size=size)
gw = Rewriter(
    model, zds, layernum, cachedir=expdir,
    low_rank_insert=True, low_rank_gradient=False,
    use_linear_insert=False,
    key_method='zca')

# Display a user interface to allow model rewriting.
savedir = f'masks/{ganname}/{modelname}'
interface = rewriteapp.GanRewriteApp(gw, size=256, mask_dir=savedir, num_canvases=32)
show(interface)